In [95]:
from bs4 import BeautifulSoup
import concurrent.futures
import csv
import pandas as pd
import random
import requests
import time

In [96]:
df = pd.read_csv("lc_output.csv")  
    
df.head(10) 

,lccn
0,https://www.loc.gov/item/sn85026945/
1,https://www.loc.gov/item/sn93067670/
2,https://www.loc.gov/item/sn93067668/
3,https://www.loc.gov/item/sn84026853/
4,https://www.loc.gov/item/sn85042527/
5,https://www.loc.gov/item/sn88064057/
6,https://www.loc.gov/item/sn83045004/
7,https://www.loc.gov/item/sn83045003/
8,https://www.loc.gov/item/sn98069055/
9,https://www.loc.gov/item/sn83016734/


In [97]:
lccns = df.lccn
lccns.head()

0    https://www.loc.gov/item/sn85026945/
1    https://www.loc.gov/item/sn93067670/
2    https://www.loc.gov/item/sn93067668/
3    https://www.loc.gov/item/sn84026853/
4    https://www.loc.gov/item/sn85042527/
Name: lccn, dtype: object

In [98]:
urls = []

for lccn in lccns:
    print(f"{lccn}?fo=json")
    urls.append(f"{lccn}?fo=json")

https://www.loc.gov/item/sn85026945/?fo=json
https://www.loc.gov/item/sn93067670/?fo=json
https://www.loc.gov/item/sn93067668/?fo=json
https://www.loc.gov/item/sn84026853/?fo=json
https://www.loc.gov/item/sn85042527/?fo=json
https://www.loc.gov/item/sn88064057/?fo=json
https://www.loc.gov/item/sn83045004/?fo=json
https://www.loc.gov/item/sn83045003/?fo=json
https://www.loc.gov/item/sn98069055/?fo=json
https://www.loc.gov/item/sn83016734/?fo=json
https://www.loc.gov/item/sn87093220/?fo=json
https://www.loc.gov/item/sn86074011/?fo=json
https://www.loc.gov/item/sn84029385/?fo=json
https://www.loc.gov/item/sn84029386/?fo=json
https://www.loc.gov/item/sn84025980/?fo=json
https://www.loc.gov/item/sn98066259/?fo=json
https://www.loc.gov/item/sn92024097/?fo=json
https://www.loc.gov/item/sn86069496/?fo=json
https://www.loc.gov/item/sn97060075/?fo=json
https://www.loc.gov/item/sn96060044/?fo=json
https://www.loc.gov/item/sn87093056/?fo=json
https://www.loc.gov/item/sn86058063/?fo=json
https://ww

https://www.loc.gov/item/sn88085488/?fo=json
https://www.loc.gov/item/sn95047324/?fo=json
https://www.loc.gov/item/sn89058133/?fo=json
https://www.loc.gov/item/sn88059517/?fo=json
https://www.loc.gov/item/sn83016922/?fo=json
https://www.loc.gov/item/sn83035526/?fo=json
https://www.loc.gov/item/sn85054616/?fo=json
https://www.loc.gov/item/sn84025829/?fo=json
https://www.loc.gov/item/2007252175/?fo=json
https://www.loc.gov/item/sn98047754/?fo=json
https://www.loc.gov/item/2007252185/?fo=json
https://www.loc.gov/item/sn86064255/?fo=json
https://www.loc.gov/item/sn86058541/?fo=json
https://www.loc.gov/item/sn86058544/?fo=json
https://www.loc.gov/item/sn86058542/?fo=json
https://www.loc.gov/item/sn86058545/?fo=json
https://www.loc.gov/item/sn86090827/?fo=json
https://www.loc.gov/item/sn86071738/?fo=json
https://www.loc.gov/item/sn82015743/?fo=json
https://www.loc.gov/item/sn88056093/?fo=json
https://www.loc.gov/item/sn85053167/?fo=json
https://www.loc.gov/item/sn89052040/?fo=json
https://ww

In [99]:
len(urls)

3683

In [100]:
# used for testing
urlss = ["https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json", "https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json", "https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json", "https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json", "https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json", "https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json"]
p = (urls[:20])
t = ["https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn84026853/?fo=json", "https://www.loc.gov/item/sn85042527/?fo=json"]

In [91]:
datas = []

def parse_json(url):
    datum = {}
    response = requests.get(url, timeout=3)
    time.sleep(2)
    json_data = response.json() if response and response.status_code == 200 else None
    if json_data and 'item' in json_data:
        time.sleep(2)
        datum['created_published'] = json_data.get('item').get('created_published', 'none')
        datum['date'] = json_data.get('item').get('date', 'none')
        datum['dates_of_publication'] = json_data.get('item').get('dates_of_publication', 'none')
        datum['description'] = json_data.get('item').get('description', 'none')
        datum['essay'] = json_data.get('item').get('essay', 'none')
        datum['essay_contributor'] = json_data.get('item').get('essay_contributor', 'none')
        datum['language'] = json_data.get('item').get('language', 'none')
        datum['latlong'] = json_data.get('item').get('latlong', 'none')
        datum['location'] = json_data.get('item').get('location', 'none')
        datum['raw_lccn'] = json_data.get('item').get('raw_lccn', 'none')
        datum['subjects'] = json_data.get('item').get('item').get('subjects', 'none')
        datum['title'] = json_data.get('item').get('item').get('title', 'none')
        datum['url'] = json_data.get('item').get('url', 'none')
        if datum not in datas:
            datas.append(datum)
        elif datum in datas:
            print(datum)
        time.sleep(random.randint(2, 4))

In [92]:
# run through individually and test
for ts in t:
    parse_json(ts)

In [93]:
datas

[]

In [39]:
MAX_THREADS = 20

def download_urls(dl_urls):
    threads = min(MAX_THREADS, len(dl_urls))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(parse_json, dl_urls)

In [40]:
def main(dl_urls):
    t0 = time.time()
    download_urls(dl_urls)
    t1 = time.time()
    print(f"{t1-t0} seconds to download {len(dl_urls)} urls.")


In [41]:
main(urls)

213.32726001739502 seconds to download 3683 urls.


In [43]:
len(datas)

45

In [44]:
datas

[{'created_published': ['Maryville, Tenn. : [publisher not identified]'],
  'date': '1879',
  'dates_of_publication': '1879-1883',
  'description': ['Weekly Began in 1879; ceased in Apr. 1883. Publishers: W.B. Scott, Sr., <1881-1882>; Democrat Pub. Co., <1882> Editors: R.N. Hood, <1880>; L.C. Blankenship, <1883> Archived issues are available in digital format as part of the Library of Congress Chronicling America online collection. Merged with: Maryville watchman, to form: East Tennessee news (Maryville, Tenn.). Description based on: Vol. 1, no. 11 (July 31, 1879). Maryville watchman (DLC)sn 96091622 (OCoLC)35116660 East Tennessee news (Maryville, Tenn.) (DLC)sn 96091624 (OCoLC)35116936'],
  'essay': '<p>\n\tIn 1847 William B. Scott, a free black, traveled to East Tennessee with his family from their home in North Carolina. After a brief stay in Knoxville, Scott moved to Blount County where he worked as a saddle and harness maker. At the start of the Civil War, Scott moved his family b

In [45]:
 with open("raw.csv", "w") as f:
        field_names = ['created_published', 'date', 'dates_of_publication', 'description', 'essay', 'essay_contributor',
                          'language', 'latlong', 'location', 'raw_lccn', 'subjects', 'title', 'url']
        writer = csv.DictWriter(f, field_names)
        writer.writerow({x: x for x in field_names})
        for row in datas:
            writer.writerow(row)


In [46]:
raw = pd.read_csv("raw.csv")  
    
raw

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Maryville, Tenn. : [publisher not identified]']",1879,1879-1883,['Weekly Began in 1879; ceased in Apr. 1883. P...,"<p>\n\tIn 1847 William B. Scott, a free black,...",['University of Tennessee'],['english'],"[35.75379, -83.9734]","['united states', 'blount county', 'maryville'...",sn 96091617,"['Maryville (Tenn.)--Newspapers', 'Blount Coun...",Blount County democrat.,https://www.loc.gov/item/sn96091617/
1,"['Bluffton, Ohio : [Bluffton News Pub. Co.]']",1875,1875-current,"['Weekly Began with July 28, 1875 issue. Descr...",<p>\n\tWhen Joseph DeFord established the vill...,"['Ohio Historical Society, Columbus, OH']",['english'],"[40.89375, -83.8914]","['united states', 'allen', 'bluffton', 'ohio']",sn 87076554,"['American newspapers--Ohio--Bluffton', 'Ameri...",The Bluffton news.,https://www.loc.gov/item/sn87076554/
2,"['Bogalusa, La. : Clyde S. Moss']",1918,1918-19??,"['4 no. per wk., <Jan. 3, 1951-Dec. 28, 1951> ...","<p>\n\tIncorporated in 1914, Bogalusa is one o...","['Louisiana State University; Baton Rouge, LA']",['english'],"[30.79062, -89.8643]","['united states', 'louisiana', 'bogalusa', 'wa...",sn 88064055,"['Bogalusa (La.)--Newspapers', 'Louisiana--Bog...",Bogalusa enterprise and American.,https://www.loc.gov/item/sn88064055/
3,"['Bogalusa, La. : Clyde S. Moss, 1914-']",1914,1914-1918,"['Weekly Vol. 1, no. 1 (Dec. 31, [1914])- Ceas...","<p>\n\tIncorporated in 1914, Bogalusa is one o...","['Louisiana State University; Baton Rouge, LA']",['english'],"[30.79062, -89.8643]","['united states', 'washington parish', 'bogalu...",sn 88064054,"['Bogalusa (La.)--Newspapers', 'Washington Par...",Bogalusa enterprise.,https://www.loc.gov/item/sn88064054/
4,"['Boise, Idaho : [F. Floed], 1906-1910.']",1906,1906-1910,"['Weekly, Nov. 3, 1908-Nov. 25, 1910 Vol. 7, n...",<p>\n\tThe original Fort Boise was abandoned i...,['Idaho State Historical Society'],['english'],"[43.60764, -116.1934]","['united states', 'boise', 'idaho', 'ada']",sn 88056027,"['Boise (Idaho)--Newspapers', 'Idaho--Boise', ...",The Boise citizen.,https://www.loc.gov/item/sn88056027/
5,"['San Juan, P.R. : Imprenta de Don Santiago Da...",18??,18??-1918,"['Daily (except Sun.), Jan. 2, 1899- Ceased in...",<p>\n\tThe <em>Bolet&#237;n mercantil de Puert...,"['University of Puerto Rico, Río Piedras Campu...",['spanish'],"[18.46633, -66.10473]","['united states', 'san juan', 'puerto rico']",sn 91099739,"['San Juan (P.R.)--Newspapers', 'Puerto Rico--...",Boletín mercantil de Puerto Rico.,https://www.loc.gov/item/sn91099739/
6,"['Bannock City, I.T. [Idaho City, Idaho] : T.J...",1863,1863-1864,"['Weekly Vol. 1, no. 1 (Sept. 29, 1863)-v. 2, ...",<p>\n\tBrothers John S. and Thomas J. Butler f...,['Idaho State Historical Society'],['english'],"[43.82964, -115.83249]","['united states', 'idaho', 'bannock city', 'id...",sn 89055016,"['Bannock City (Idaho)--Newspapers', 'Idaho Ci...",Boise news.,https://www.loc.gov/item/sn89055016/
7,"['Bolivar, Tenn. : Green & Williams, 1888-1946.']",1888,1888-1946,"['Weekly Vol. 24, no. 15 (Nov. 30, 1888)-v. 78...",<p>\n\tThe first issue of the <em>Bolivar Bull...,['University of Tennessee'],['english'],"[35.2562, -88.98898]","['united states', 'bolivar', 'tennessee', 'har...",sn 89058007,"['Bolivar (Tenn.)--Newspapers', 'Hardeman Coun...",The Bolivar bulletin.,https://www.loc.gov/item/sn89058007/
8,"['Bolivar, Hardeman County, Tenn. : J.B. Mille...",1865,1865-1888,"['Weekly Began in Aug. 1865? -v. 23, no. 34 (A...",<p>\n\tThe first issue of the <em>Bolivar Bull...,['University of Tennessee'],['english'],"[35.2562, -88.98898]","['united states', 'bolivar', 'tennessee', 'har...",sn 85033306,"['Bolivar (Tenn.)--Newspapers', 'Hardeman Coun...",The Bolivar bulletin.,https://www.loc.gov/item/sn85033306/
9,"['Tabor, S.D. : Independent Pub. Co., 1904-191...",1904,1904-1913,"['Weekly Vol. 1, no. 1 (Oct. 6, 1904)-v. 9, no.

In [ ]:
# strip out html tags

# Function to remove tags
def remove_tags(html):
  
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
  
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
  
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)
  
# Print the extracted data
# return remove_tags(raw))

In [ ]:
for urlt in raw:
    print(urlt)

# todo:
0. write this right.
1. check for and delete csv if does exist
2. refactor / rate limit api calls.
3. write to list or pickle before writing to csv
4. test which sleep function is necessary
5. document code
6. NER / BERT
7. Begin planning for fall / next steps
8. Ingest / Clean xlsx